## This notebook shows a study of using the scatter in the centroid between different bands to look for effect of blending

In [ ]:
#### Do the usual imports

In [ ]:
import tables_io
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#### Change this to be the root of the current PZ working area

In [ ]:
# pz_dir = '/global/cfs/cdirs/lsst/groups/PZ/DP1'
# if that fails you can use this
# pz_dir = '/global/u2/e/echarles/dx'
pz_dir = '/Users/echarles/pz'

In [ ]:
#### Get the data. 

In [ ]:
t = tables_io.read(f'{pz_dir}/data/dp1_v29.0.0/5063/object.parquet')

#### Extract the centroids and the blendedness

In [ ]:
blendedness = t['i_blendedness'].to_numpy()
blendedness = np.where(np.isnan(blendedness), 0., blendedness).clip(0., 1.)
mask  = np.bitwise_and(
    np.bitwise_and(
        t['i_extendedness'].to_numpy() == 1,
        t['g_psfFlux'].to_numpy() / t['g_psfFluxErr'].to_numpy() > 10,
    ),
    np.bitwise_and(
        t['i_psfFlux'].to_numpy() / t['i_psfFluxErr'].to_numpy() > 10,
        t['r_psfFlux'].to_numpy() / t['r_psfFluxErr'].to_numpy() > 10,
    ),
)
mask_unblend = np.bitwise_and(mask, blendedness<0.1)
mask_blend = np.bitwise_and(mask, blendedness>0.1)
g_ra = t['g_ra'].to_numpy()
g_raErr = t['g_raErr'].to_numpy()
g_dec = t['g_dec'].to_numpy()
g_decErr = t['g_decErr'].to_numpy()
cos_g_dec = np.cos(np.radians(g_dec))
cos_g_raErr = cos_g_dec*g_raErr
r_ra = t['r_ra'].to_numpy()
r_raErr = t['r_raErr'].to_numpy()
r_dec = t['r_dec'].to_numpy()
r_decErr = t['r_decErr'].to_numpy()
cos_r_dec = np.cos(np.radians(r_dec))
i_ra = t['i_ra'].to_numpy()
i_raErr = t['i_raErr'].to_numpy()
i_dec = t['i_dec'].to_numpy()
i_decErr = t['i_decErr'].to_numpy()
cos_i_dec = np.cos(np.radians(i_dec))
z_ra = t['z_ra'].to_numpy()
z_raErr = t['z_raErr'].to_numpy()
z_dec = t['z_dec'].to_numpy()
z_decErr = t['z_decErr'].to_numpy()
cos_z_dec = np.cos(np.radians(z_dec))
ra_vals = np.vstack([g_ra*cos_g_dec, r_ra*cos_r_dec, i_ra*cos_i_dec, z_ra*cos_z_dec])
dec_vals = np.vstack([g_dec, r_dec, i_dec, z_dec])
errs = np.sqrt(cos_g_raErr*cos_g_raErr+g_decErr*g_decErr)

#### compute thet scatter in the centroids

In [ ]:
cent_scatter = np.sqrt(np.var(ra_vals, axis=0) + np.var(dec_vals, axis=0))
cent_scatter_scaled = np.sqrt(np.var(ra_vals, axis=0) + np.var(dec_vals, axis=0))/errs

#### Plot the scaled scatter in the centroids

In [ ]:
_ = plt.hist(cent_scatter_scaled[mask_unblend], bins=np.linspace(0, 5, 51), label='unblended')
_ = plt.hist(cent_scatter_scaled[mask_blend], bins=np.linspace(0, 5, 51), label='blended')
_ = plt.xlabel(r'Scaled scatter: $\sigma_{\rm cent} / \delta_{\rm cent}$')
_ = plt.ylabel('Objects / 0.1')
_ = plt.legend()

In [ ]:
_ = plt.hist(3600*cent_scatter[mask_unblend], bins=np.linspace(0, 0.2, 201), label='unblended')
_ = plt.hist(3600*cent_scatter[mask_blend], bins=np.linspace(0, 0.2, 201), label='blended')
_ = plt.xlabel(r'Scatter: $\sigma_{\rm cent}$ ["]')
_ = plt.yscale('log')
_ = plt.ylabel('Objects / 1.0 mas')
_ = plt.legend()

In [ ]:
_ = plt.hist(3600*errs[mask_unblend], bins=np.linspace(0, 1.0, 101), label='unblended')
_ = plt.hist(3600*errs[mask_blend], bins=np.linspace(0, 1.0, 101), label='blended')
_ = plt.xlabel(r'Reported errors: $\delta_{\rm cent}$ ["]')
_ = plt.yscale('log')
_ = plt.ylabel('Objects / 0.01 mas')
_ = plt.legend()